In [45]:
from __future__ import division
from math import log, exp
from random import random
import numpy as np

COUNT = 0

In [41]:
LFMEMO = {}
def logfact(n):
    # return ln(n!)
    if n in LFMEMO:
        return LFMEMO[n]
    lf = sum(log(i) for i in range(1, n+1))
    FMEMO[n] = lf
    return lf

TMEMO = {}
def tutte(n, m):
    global COUNT
    COUNT = COUNT + 1
    if n < 0 or m < 0:
        return 0
    if (n, m) in TMEMO:
        return TMEMO[(n, m)]
    #val = 2**(n+1)*fact(2*m+1)*fact(2*m+3*n)/(fact(m)**2*fact(n)*fact(2*m+2*n+2))
    val = (n+1)*log(2) + logfact(2*m+1) + logfact(2*m+3*n)
    val -= logfact(m)*2 + logfact(n) + logfact(2*m+2*n+2)
    val = exp(val)
    TMEMO[(n, m)] = val
    return val

def approx(n, m):
    try:
        return tutte(n, m)/(factorial(2*m)/factorial(m)**2/(m+1)*13.5**n/n**2.5)
    except:
        return 0

In [6]:
def rtutte(n, m):
    total = 0
    if n > 0:
        val = tutte(n-1, m+1)
        print('inner', val)
        total += val
    for j in range(0, m):
        for k in range(0, n+1):
            a = tutte(k, j)
            b = tutte(n-k, m-j-1)
            print((k, j), (n-k, m-j-1), a, b, a*b)
            total += a*b
    return total

In [11]:
def sample(N):
    probs = []
    sizes = []
    for n in range(0, N-1):
        m = N - n - 2
        probs.append(tutte(n, m))
        sizes.append((n, m))
    total = sum(probs)
    probs = [p/total for p in probs]
    tot = 0
    for i in range(len(probs)):
        tot += probs[i]
    n, m = sizes[np.random.choice(N-1, p=probs)]
    return n, m

In [12]:
def make_graph(G, internal, external):
    n = len(internal)
    m = len(external)-2
    if n == 0 and m == 0:
        # nothing else to add
        return
    total_graphs = tutte(n, m)
    add_internal = tutte(n-1, m+1)
    r = random()
    if r < add_internal / total_graphs:
        # add internal triangle
        triangle = (external[0], external[1], internal[0])
        G.append(triangle)
        make_graph(G, internal[1:], external + [internal[0]])
    else:
        remaining_graphs = total_graphs - add_internal
        counts = []
        choices = []
        for j in range(0, m):
            for k in range(0, n+1):
                count = tutte(k, j) * tutte(n-k, m-j-1)
                counts.append(count/remaining_graphs)
                # there are [count] triangulations with k internal and j+2 external vertices on one side and the rest on the other
                choices.append((k, j))
        choice = choices[np.random.choice(len(choices), p=counts)]
        k, j = choice
        triangle = (external[0], external[1], external[j+2])
        G.append(triangle)
        left = external[1:j+3]
        right = external[j+2:] + [external[0]]
        make_graph(G, internal[:k], left)
        make_graph(G, internal[k:], right)
    return G

In [44]:
# show that the number of graphs generated by this algorithm is the same(ish) as T(n, m)
graphs = set()
for _ in range(100000):
    n = 4
    m = 2
    internal = range(m+2, m+n+2)
    external = range(0, m+2)
    G = []
    graphs.add(tuple(make_graph(G, internal, external)))
G = []
G = make_graph(G, internal, external)
print len(graphs), tutte(n, m)

KeyboardInterrupt: 

In [80]:
COUNT = 0
SIZE = 200
for _ in range(500):
    G = []
    n, m = sample(SIZE)
    internal = range(m+2, m+n+2)
    external = range(0, m+2)
    G = len(make_graph(G, internal, external))
print COUNT/200

382851.82


In [48]:
COUNT

207171